In [2]:
import pandas as pd
import networkx as nx
from collections import defaultdict
import gzip
import os
import numpy as np
from scipy.sparse import csr_matrix
from node2vec import Node2Vec

/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set base directory
base_dir = '/Users/terryma/Documents/physionet.org/files/mimiciii/1.4'

# Load relevant tables with correct paths
patients = pd.read_csv(gzip.open(os.path.join(base_dir, 'PATIENTS.csv.gz'), 'rt'), 
                      usecols=['SUBJECT_ID'])
admissions = pd.read_csv(gzip.open(os.path.join(base_dir, 'ADMISSIONS.csv.gz'), 'rt'), 
                        usecols=['SUBJECT_ID', 'HADM_ID'])
diagnoses = pd.read_csv(gzip.open(os.path.join(base_dir, 'DIAGNOSES_ICD.csv.gz'), 'rt'), 
                       usecols=['HADM_ID', 'ICD9_CODE'])
procedures = pd.read_csv(gzip.open(os.path.join(base_dir, 'PROCEDURES_ICD.csv.gz'), 'rt'), 
                        usecols=['HADM_ID', 'ICD9_CODE'])
prescriptions = pd.read_csv(gzip.open(os.path.join(base_dir, 'PRESCRIPTIONS.csv.gz'), 'rt'), 
                          usecols=['HADM_ID', 'NDC'])
labevents = pd.read_csv(gzip.open(os.path.join(base_dir, 'LABEVENTS.csv.gz'), 'rt'), 
                       usecols=['HADM_ID', 'ITEMID'])

# Add some basic data validation and progress printing
print("Loading data...")
print(f"Patients: {len(patients)} records")
print(f"Admissions: {len(admissions)} records")
print(f"Diagnoses: {len(diagnoses)} records")
print(f"Procedures: {len(procedures)} records")
print(f"Prescriptions: {len(prescriptions)} records")
print(f"Lab Events: {len(labevents)} records")

# Create visit-level records with progress updates
print("\nCreating visit-level records...")
visit_codes = defaultdict(set)

print("Processing diagnoses...")
for _, row in diagnoses.iterrows():
    if pd.notna(row['ICD9_CODE']):  # Skip null values
        visit_codes[row['HADM_ID']].add(f'DIAG_{row["ICD9_CODE"]}')

print("Processing procedures...")
for _, row in procedures.iterrows():
    if pd.notna(row['ICD9_CODE']):
        visit_codes[row['HADM_ID']].add(f'PROC_{row["ICD9_CODE"]}')

print("Processing prescriptions...")
for _, row in prescriptions.iterrows():
    if pd.notna(row['NDC']):
        visit_codes[row['HADM_ID']].add(f'MED_{row["NDC"]}')

print("Processing lab events...")
for _, row in labevents.iterrows():
    if pd.notna(row['ITEMID']):
        visit_codes[row['HADM_ID']].add(f'LAB_{row["ITEMID"]}')

# Build the graph with progress updates
print("\nBuilding graph...")
H = nx.Graph()

print("Adding nodes...")
all_codes = set(code for codes in visit_codes.values() for code in codes)
H.add_nodes_from(all_codes)

print("Adding edges...")
for visit_id, codes in visit_codes.items():
    codes_list = list(codes)
    for i in range(len(codes_list)):
        for j in range(i+1, len(codes_list)):
            H.add_edge(codes_list[i], codes_list[j])

# Print detailed statistics
print("\nGraph Statistics:")
print(f"Number of nodes (medical codes): {H.number_of_nodes()}")
print(f"Number of edges (connections): {H.number_of_edges()}")
print(f"Number of visits processed: {len(visit_codes)}")

# Additional analysis
avg_degree = sum(dict(H.degree()).values()) / H.number_of_nodes()
print(f"Average node degree: {avg_degree:.2f}")

# Print code type distribution
code_types = defaultdict(int)
for node in H.nodes():
    prefix = node.split('_')[0]
    code_types[prefix] += 1

print("\nCode type distribution:")
for prefix, count in code_types.items():
    print(f"{prefix}: {count} codes")

Loading data...
Patients: 46520 records
Admissions: 58976 records
Diagnoses: 651047 records
Procedures: 240095 records
Prescriptions: 4156450 records
Lab Events: 27854055 records

Creating visit-level records...
Processing diagnoses...
Processing procedures...
Processing prescriptions...
Processing lab events...

Building graph...
Adding nodes...
Adding edges...

Graph Statistics:
Number of nodes (medical codes): 13923
Number of edges (connections): 9129990
Number of visits processed: 5667981
Average node degree: 1311.50

Code type distribution:
PROC: 2009 codes
DIAG: 6984 codes
MED: 4204 codes
LAB: 726 codes


In [7]:
import numpy as np
import torch
from torch_geometric.nn import Node2Vec
import torch_geometric.transforms as T
from torch_geometric.utils import from_networkx

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create output directory if it doesn't exist
output_dir = '/Users/terryma/Documents'

# 1. Generate edge labels (can be parallelized with CUDA)
print("Generating edge labels...")
edge_labels = []
edges = list(H.edges())
# Convert to torch tensors for GPU processing
edge_tensor = torch.tensor([[u, v] for u, v in edges]).to(device)

def compute_edge_labels(edge_tensor, H):
    labels = []
    for u, v in edge_tensor.cpu().numpy():
        common_neighbors = len(set(H.neighbors(u)).intersection(set(H.neighbors(v))))
        label = 1 if common_neighbors > np.mean([H.degree(u), H.degree(v)])/2 else 0
        labels.append(label)
    return torch.tensor(labels, device=device)

edge_labels = compute_edge_labels(edge_tensor, H)

# Save edge labels
edge_labels_path = os.path.join(output_dir, 'edge-labels-mimic3.txt')
np.savetxt(edge_labels_path, edge_labels.cpu().numpy(), fmt='%d', delimiter=',')

# 2. Generate hyperedges
print("Generating hyperedges...")
node_to_index = {node: idx for idx, node in enumerate(H.nodes())}
hyperedges = []
for visit_id, codes in visit_codes.items():
    code_indices = [node_to_index[code] for code in codes]
    hyperedges.append(code_indices)

# Save hyperedges
hyperedges_path = os.path.join(output_dir, 'hyperedges-mimic3.txt')
with open(hyperedges_path, 'w') as f:
    for edge in hyperedges:
        f.write(','.join(map(str, edge)) + '\n')

# 3. Generate node embeddings using GPU-accelerated node2vec
print("Generating node embeddings...")
# Convert networkx graph to PyTorch Geometric data
data = from_networkx(H)
data = data.to(device)

# Initialize GPU-accelerated Node2Vec
model = Node2Vec(
    data.edge_index,
    embedding_dim=64,
    walk_length=30,
    context_size=10,
    walks_per_node=200,
    num_negative_samples=1,
    p=1,
    q=1,
    sparse=True
).to(device)

# Train using GPU
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    total_loss = 0
    for _ in range(100):  # num epochs
        optimizer.zero_grad()
        loss = model.loss()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / 100

print("Training node embeddings...")
for epoch in range(30):
    loss = train()
    if (epoch + 1) % 5 == 0:
        print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')

# Get embeddings
@torch.no_grad()
def get_embeddings():
    model.eval()
    z = model()
    return z.cpu().numpy()

embeddings = get_embeddings()

# Save node embeddings
embeddings_path = os.path.join(output_dir, 'node-embeddings-mimic3')
np.savetxt(embeddings_path, embeddings)

print(f"\nFiles saved to:")
print(f"Edge labels: {edge_labels_path}")
print(f"Hyperedges: {hyperedges_path}")
print(f"Node embeddings: {embeddings_path}")

Generating edge labels...


KeyboardInterrupt: 